In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])
#user_app_usage = pd.read_csv("user_app_usage.csv")
app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])

In [4]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    cats = list(set(app_info['category']))
    category2id = dict(zip(sorted(cats), range(0,len(cats))))
    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid')    

            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']


            df = df.merge(app_info, how='left', on='appId')
            now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame(), how='left', on='uid')
            #print(df.groupby(['uid'])['category'].value_counts().index[0])
            now_df['usage_most_used_category'] = df.groupby(['uid'])['category'].transform(f)
            resTable = pd.concat([resTable, now_df])
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [5]:
processUserAppUsage()

index: 1


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 101
index:

In [4]:
#将user_basic_info.csv 和 user_behavior_info.csv中的字符值编码成可以训练的数值类型，合并
class2id = {}
id2class = {}
def mergeBasicTables(baseTable):
    resTable = baseTable.merge(user_basic_info, how='left', on='uid', suffixes=('_base0', '_ubaf'))
    resTable = resTable.merge(user_behavior_info, how='left', on='uid', suffixes=('_base1', '_ubef'))
    cat_columns = ['city','prodName','color','carrier','os','ct']
    for c in cat_columns:
        resTable[c] = resTable[c].apply(lambda x: x if type(x)==str else str(x))
        sort_temp = sorted(list(set(resTable[c])))  
        class2id[c+'2id'] = dict(zip(sort_temp, range(1, len(sort_temp)+1)))
        id2class['id2'+c] = dict(zip(range(1,len(sort_temp)+1), sort_temp))
        resTable[c] = resTable[c].apply(lambda x: class2id[c+'2id'][x])
        
    return resTable

In [5]:
#处理app使用相关数据
#对user_app_actived.csv简单统计
#将之前训练的appuseProcess.csv进行合并
def mergeAppData(baseTable):
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    resTable['appId'] = resTable['appId'].apply(lambda x: len(list(x.split('#'))))
    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    resTable = resTable.fillna(0)
    #print(resTable[:5])
    return resTable

In [6]:
#合并用户基本特征以及app使用相关特征，作为训练集和测试集
df_train = mergeAppData(mergeBasicTables(age_train))
df_test = mergeAppData(mergeBasicTables(age_test))
print(df_train.shape)
print(df_test.shape)

(2010000, 34)
(502500, 33)


In [27]:
df_train.to_csv("df_train.csv")
df_test.to_csv("df_test.csv")

In [17]:
print(df_train.dtypes)

uid                           int64
age_group                     int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               float64
duration_max                

In [18]:
#训练模型

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [21]:
print("训练模型：")
param = {
        'learning_rate': 1e-3,
        'num_boost_round': 100,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 31,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 255,
        'feature_fraction': 0.9,
        'metric': 'multi_error',
        'boosting': 'dart',
        'device':'gpu',
        'num_threads': 12
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    
    
    model = lgb.train(param, train, valid_sets=[vali])
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    
    
    y_test = model.predict(test,num_iteration=model.best_iteration)
    model.save_model('model.txt')
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

训练模型：
0
training start...


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_error: 0.701493
[2]	valid_0's multi_error: 0.701493
[3]	valid_0's multi_error: 0.701493
[4]	valid_0's multi_error: 0.701493
[5]	valid_0's multi_error: 0.701493
[6]	valid_0's multi_error: 0.701493
[7]	valid_0's multi_error: 0.701493
[8]	valid_0's multi_error: 0.701493
[9]	valid_0's multi_error: 0.701493
[10]	valid_0's multi_error: 0.701493
[11]	valid_0's multi_error: 0.701493
[12]	valid_0's multi_error: 0.701493
[13]	valid_0's multi_error: 0.701493
[14]	valid_0's multi_error: 0.701493
[15]	valid_0's multi_error: 0.701493
[16]	valid_0's multi_error: 0.701493
[17]	valid_0's multi_error: 0.701493
[18]	valid_0's multi_error: 0.701493
[19]	valid_0's multi_error: 0.701493
[20]	valid_0's multi_error: 0.701493
[21]	valid_0's multi_error: 0.701493
[22]	valid_0's multi_error: 0.701493
[23]	valid_0's multi_error: 0.701493
[24]	valid_0's multi_error: 0.701493
[25]	valid_0's multi_error: 0.701493
[26]	valid_0's multi_error: 0.701493
[27]	valid_0's multi_error: 0.701493
[28]	valid

KeyboardInterrupt: 

In [13]:
param = {
        'learning_rate': 1e-5,
        'num_boost_round': 3000,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 31,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 63,
        'feature_fraction': 0.8,
        'metric': 'multi_error',
        'boosting': 'dart',
        'device':'gpu',
        'num_threads': 8
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
classifier = lgb.LGBMClassifier(learning_rate=0.05, num_leaves=63, max_depth=16, objective='multiclass', silent=False, reg_alpha=0.1, reg_lambda=0.2, n_estimators=6000)

In [14]:
classifier.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.05, max_depth=16,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=6000, n_jobs=-1, num_leaves=63,
        objective='multiclass', random_state=None, reg_alpha=0.1,
        reg_lambda=0.2, silent=False, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [12]:
y_test = classifier.predict(test)
#y_test = [np.argmax(x) for x in y_test]
cv_pred = np.array(y_test).reshape(-1, 1)
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [19]:
y_test = classifier.predict(test)
#y_test = [np.argmax(x) for x in y_test]
cv_pred = np.array(y_test).reshape(-1, 1)
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [ ]:
from sklearn.ensemble import RandomForestClassifier  
rfc = RandomForestClassifier(n_estimators=2000)
rfc.fit(X, y)

In [8]:
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [9]:
vali = df_train.sample(n=20000, replace=False, axis=0)

In [9]:
import xgboost as xgb

In [10]:
model = xgb.XGBClassifier(max_depth=10, learning_rate=0.1, n_jobs=16, n_estimators=1000, silent=True, scale_pos_weight=20, obgective='multi:softmax', num_class=7, early_stopping_rounds=200, min_child_weight=4)

In [14]:
Xt = vali.drop(['age_group','uid'], axis=1)
yt = vali['age_group']
X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']

In [ ]:
model.fit(X, y, eval_set=[(Xt, yt)], xgb_model=model.get_booster(), verbose=True)

In [20]:
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

In [21]:
y_test = model.predict(test)
#y_test = [np.argmax(x) for x in y_test]
cv_pred = np.array(y_test).reshape(-1, 1)
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [16]:
model.save_model('xgb.model')

In [53]:
age_train['age_group'].nunique()

6

In [11]:
modelbst = xgb.Booster(model_file='xgb.model')

In [12]:
model = xgb.XGBClassifier(max_depth=10, learning_rate=0.1, n_estimators=1000, n_jobs=16, silent=True, scale_pos_weight=20, obgective='multi:softmax', num_class=7, early_stopping_rounds=200, min_child_weight=4)

In [13]:
X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

In [16]:
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    
    
    model.fit(x_train, y_train, eval_set=[(x_vali, y_vali)], xgb_model=modelbst, verbose=True)
    model.save_model('xgb.model')
    modelbst = model.get_booster()
    

        


0
training start...
[0]	validation_0-merror:0.365941
[1]	validation_0-merror:0.365974
[2]	validation_0-merror:0.366001
[3]	validation_0-merror:0.366037
[4]	validation_0-merror:0.366104
[5]	validation_0-merror:0.366086
[6]	validation_0-merror:0.366107
[7]	validation_0-merror:0.366168
[8]	validation_0-merror:0.366116
[9]	validation_0-merror:0.366126
[10]	validation_0-merror:0.366186
[11]	validation_0-merror:0.366185
[12]	validation_0-merror:0.366222
[13]	validation_0-merror:0.366258
[14]	validation_0-merror:0.366262
[15]	validation_0-merror:0.366286
[16]	validation_0-merror:0.366291
[17]	validation_0-merror:0.366261
[18]	validation_0-merror:0.366317
[19]	validation_0-merror:0.366307
[20]	validation_0-merror:0.366332
[21]	validation_0-merror:0.366428
[22]	validation_0-merror:0.366432
[23]	validation_0-merror:0.366435
[24]	validation_0-merror:0.366525
[25]	validation_0-merror:0.366561
[26]	validation_0-merror:0.366582
[27]	validation_0-merror:0.366628
[28]	validation_0-merror:0.366616
[29]

[238]	validation_0-merror:0.371708
[239]	validation_0-merror:0.371701
[240]	validation_0-merror:0.37171
[241]	validation_0-merror:0.371771
[242]	validation_0-merror:0.371801
[243]	validation_0-merror:0.371825
[244]	validation_0-merror:0.371792
[245]	validation_0-merror:0.371807
[246]	validation_0-merror:0.37174
[247]	validation_0-merror:0.371768
[248]	validation_0-merror:0.371786
[249]	validation_0-merror:0.371847
[250]	validation_0-merror:0.371837
[251]	validation_0-merror:0.371855
[252]	validation_0-merror:0.371874
[253]	validation_0-merror:0.371932
[254]	validation_0-merror:0.371947
[255]	validation_0-merror:0.371983
[256]	validation_0-merror:0.371956
[257]	validation_0-merror:0.371962
[258]	validation_0-merror:0.372005
[259]	validation_0-merror:0.372004
[260]	validation_0-merror:0.372047
[261]	validation_0-merror:0.372074
[262]	validation_0-merror:0.372073
[263]	validation_0-merror:0.372102
[264]	validation_0-merror:0.372137
[265]	validation_0-merror:0.372123
[266]	validation_0-mer

[473]	validation_0-merror:0.376625
[474]	validation_0-merror:0.376656
[475]	validation_0-merror:0.376685
[476]	validation_0-merror:0.376674
[477]	validation_0-merror:0.376689
[478]	validation_0-merror:0.376716
[479]	validation_0-merror:0.37674
[480]	validation_0-merror:0.376747
[481]	validation_0-merror:0.376793
[482]	validation_0-merror:0.37678
[483]	validation_0-merror:0.376783
[484]	validation_0-merror:0.376787
[485]	validation_0-merror:0.37684
[486]	validation_0-merror:0.37685
[487]	validation_0-merror:0.376874
[488]	validation_0-merror:0.37692
[489]	validation_0-merror:0.376946
[490]	validation_0-merror:0.376958
[491]	validation_0-merror:0.376911
[492]	validation_0-merror:0.376955
[493]	validation_0-merror:0.376983
[494]	validation_0-merror:0.376983
[495]	validation_0-merror:0.376993
[496]	validation_0-merror:0.377004
[497]	validation_0-merror:0.376998
[498]	validation_0-merror:0.377031
[499]	validation_0-merror:0.377049
[500]	validation_0-merror:0.377119
[501]	validation_0-merror

[708]	validation_0-merror:0.381834
[709]	validation_0-merror:0.381856
[710]	validation_0-merror:0.38187
[711]	validation_0-merror:0.38185
[712]	validation_0-merror:0.381867
[713]	validation_0-merror:0.381892
[714]	validation_0-merror:0.381922
[715]	validation_0-merror:0.381925
[716]	validation_0-merror:0.381938
[717]	validation_0-merror:0.381928
[718]	validation_0-merror:0.381932
[719]	validation_0-merror:0.381973
[720]	validation_0-merror:0.381989
[721]	validation_0-merror:0.382016
[722]	validation_0-merror:0.381987
[723]	validation_0-merror:0.381983
[724]	validation_0-merror:0.382037
[725]	validation_0-merror:0.382032
[726]	validation_0-merror:0.382061
[727]	validation_0-merror:0.382079
[728]	validation_0-merror:0.382082
[729]	validation_0-merror:0.382114
[730]	validation_0-merror:0.382122
[731]	validation_0-merror:0.382119
[732]	validation_0-merror:0.382111
[733]	validation_0-merror:0.382101
[734]	validation_0-merror:0.382122
[735]	validation_0-merror:0.382132
[736]	validation_0-mer

[943]	validation_0-merror:0.386229
[944]	validation_0-merror:0.386249
[945]	validation_0-merror:0.386295
[946]	validation_0-merror:0.386347
[947]	validation_0-merror:0.386411
[948]	validation_0-merror:0.386431
[949]	validation_0-merror:0.386437
[950]	validation_0-merror:0.386444
[951]	validation_0-merror:0.386471
[952]	validation_0-merror:0.38649
[953]	validation_0-merror:0.386476
[954]	validation_0-merror:0.386517
[955]	validation_0-merror:0.386464
[956]	validation_0-merror:0.386465
[957]	validation_0-merror:0.38649
[958]	validation_0-merror:0.386505
[959]	validation_0-merror:0.386516
[960]	validation_0-merror:0.386543
[961]	validation_0-merror:0.386568
[962]	validation_0-merror:0.386573
[963]	validation_0-merror:0.386619
[964]	validation_0-merror:0.386637
[965]	validation_0-merror:0.386656
[966]	validation_0-merror:0.386685
[967]	validation_0-merror:0.386647
[968]	validation_0-merror:0.386674
[969]	validation_0-merror:0.386661
[970]	validation_0-merror:0.386631
[971]	validation_0-mer

KeyboardInterrupt: 

In [ ]:
model.save_model('xgb.model')

In [11]:
history

import pandas as pd
import numpy as np
import lightgbm as lgb
pd.set_option('display.max_columns',None)
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])
#user_app_usage = pd.read_csv("user_app_usage.csv")
app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.